In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import PCA, StandardScaler
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from PIL import Image
import numpy as np
import io
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.types import ArrayType, FloatType
import cv2

In [6]:
if SparkSession.builder.getOrCreate().sparkContext:
    SparkSession.builder.getOrCreate().sparkContext.stop()

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test17-spark') \
    .getOrCreate()


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 56834)
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/pyspark/a

In [7]:
PATH = os.getcwd()
PATH_Data = PATH+'/data/Test1'
PATH_Result = PATH+'/data/results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

PATH:        /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train
PATH_Data:   /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train/data/Test1
PATH_Result: /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train/data/results


In [8]:
df = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)
df = df.withColumn('label', element_at(split(df['path'], '/'),-2))

In [9]:
df.show()

+--------------------+-------------------+------+--------------------+--------------+
|                path|   modificationTime|length|             content|         label|
+--------------------+-------------------+------+--------------------+--------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5602|[FF D8 FF E0 00 1...|Apple Braeburn|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5599|[FF D

In [10]:
from pyspark.sql.types import ArrayType, IntegerType

def process_image(content):
    img = Image.open(io.BytesIO(content)).convert('L')
    return np.array(img).flatten().tolist()

process_image_udf = udf(process_image, ArrayType(IntegerType()))
df_final = df.withColumn("features", process_image_udf(df.content))

df_final.show()

+--------------------+-------------------+------+--------------------+--------------+--------------------+
|                path|   modificationTime|length|             content|         label|            features|
+--------------------+-------------------+------+--------------------+--------------+--------------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:2

In [11]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors, VectorUDT

def sparse_vector(features):
    # Assurez-vous que 'features' est sous une forme appropriée pour Vectors.sparse
    return Vectors.sparse(len(features), list(enumerate(features)))

sparse_vector_udf = udf(sparse_vector, VectorUDT())

df_X = df_final.withColumn("dense_features", sparse_vector_udf(df_final["features"]))

df_X.printSchema()
df_X.show()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- dense_features: vector (nullable = true)



+--------------------+-------------------+------+--------------------+--------------+--------------------+--------------------+
|                path|   modificationTime|length|             content|         label|            features|      dense_features|
+--------------------+-------------------+------+--------------------+--------------+--------------------+--------------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|(10000,[0,1,2,3,4...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|(10000,[0,1,2,3,4...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|(10000,[0,1,2,3,4...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2...|(10000,[0,1,2,3,4...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|Apple Braeburn|[255, 255, 255, 2..

In [12]:
pca = PCA(k=20, inputCol="dense_features", outputCol="pcaFeatures")
model = pca.fit(df_X)

24/05/29 10:38:55 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/29 10:43:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
result = model.transform(df_X).select("pcaFeatures")
result.show

<bound method DataFrame.show of DataFrame[pcaFeatures: vector]>

In [16]:
result.show()

24/05/29 13:20:44 WARN DAGScheduler: Broadcasting large task binary with size 1600.9 KiB


+--------------------+
|         pcaFeatures|
+--------------------+
|[4052.21102227018...|
|[3759.00654594035...|
|[4110.60921359289...|
|[3841.51486007574...|
|[3785.66382128323...|
|[3808.04396930490...|
|[4082.41407330736...|
|[4055.77395481855...|
|[3829.41470077008...|
|[3808.50068703526...|
|[4069.67933019970...|
|[4096.43655245458...|
|[3837.21749450920...|
|[3847.42545323123...|
|[4125.15340088608...|
|[4157.19411348962...|
|[3777.33960344046...|
|[3790.23653613181...|
|[3997.13707953211...|
|[3855.36408997278...|
+--------------------+
only showing top 20 rows



24/05/29 23:31:17 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3665484 ms exceeds timeout 120000 ms
24/05/29 23:31:17 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/29 23:31:17 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$